# Context-Aware Recommendation Algorithm by PAPERDAA-2

In [1]:
import numpy as np
import pandas as pd
import math
from sklearn import preprocessing
from itertools import product as cartesian_product

# Training

## Program Arguments

In [2]:
LMAX = 10    # Maximum transitive path length
K = 15       # Nearest neighbors

## Preprocessing

In [3]:
df = pd.read_csv('training_dataset.csv')

df[df['userid']==1066]

,userid,itemid,rating,Time,Location,Companion
221,1066,tt0120912,5,Weekend,Home,Family
222,1066,tt1657301,2,Weekend,Home,Family
223,1066,tt0211915,1,Weekend,Home,Family
224,1066,tt1478338,1,Weekend,Home,Family
225,1066,tt0133093,2,Weekend,Home,Family
226,1066,tt0120912,5,Weekday,Cinema,Family
227,1066,tt1657301,2,Weekday,Cinema,Family
228,1066,tt0211915,1,Weekday,Cinema,Family
229,1066,tt1478338,1,Weekday,Cinema,Family
230,1066,tt0133093,2,Weekday,Cinema,Family


In [4]:
encoder = preprocessing.LabelEncoder()

user_item_context_encodings = []
user_item_context_reverse_encodings = []
maximum_rating = df[df.columns[2]].max()

encoded_df = df.copy()
# Encode userid, itemid, and contextual informations for item splitting
for column_index in range(len(df.columns)):
    
    # Column attribute is not rating
    if column_index != 2:
        
        # Fit encoder
        encoder.fit(df[df.columns[column_index]])
        encoded_df[df.columns[column_index]] = encoder.transform(
                df[df.columns[column_index]]
            )
    
    # Column is nor user or rating
    if column_index != 2:
            user_item_context_encodings.append(
                dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
            )
            user_item_context_reverse_encodings.append(
                dict(zip(encoder.transform(encoder.classes_), encoder.classes_))
            )

In [5]:
display(encoded_df)

,userid,itemid,rating,Time,Location,Companion
0,2,38,1,0,0,0
1,2,8,1,0,0,0
2,2,74,1,0,0,0
3,2,5,1,0,0,0
4,2,2,1,0,0,0
...,...,...,...,...,...,...
3432,65,35,1,1,1,2
3433,65,62,2,1,1,2
3434,65,25,1,1,1,2
3435,65,50,1,1,1,2


## Item Splitting

In [6]:
# Cartesian product all items and contexts

users = user_item_context_encodings[0].values()
items = user_item_context_encodings[1].values()
contexts = [
    context_trans.values() for context_trans in user_item_context_encodings[2:]
]

context_T = list(cartesian_product(items, *contexts))

In [7]:
# Generate new user-item matrix for new items
rating_matrix = np.zeros((len(users), len(context_T)), dtype=object)

In [8]:
for row in encoded_df.iterrows():
    data = tuple(row[1])
    user = data[0]
    item = data[1]
    rating = data[2]
    context_item = (item, *data[3:])
    
    index = context_T.index(context_item)
    
    rating_matrix[user][index] = int(rating)/int(maximum_rating)

In [9]:
display(rating_matrix)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0.2, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1.0, 0.8, 1.0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

## Graph Similarity Calculation (Item-Based)

In [10]:
def get_column_vector(matrix, index):
    column_vector = []

    for row in matrix:
        column_vector.append(row[index])
    return column_vector


def dot_product(c1, c2):
    result = 0
    if (len(c1) == len(c2)):
        for i in range(len(c1)):
            result += c1[i] * c2[i]

    return result


def vector_length(vector):
    result = 0

    for integer in vector:
        result += integer * integer
    
    result = result**0.5
    return result if result != 0 else False


def cosine_similarity(c1, c2):
    l1 = vector_length(c1) 
    l2 = vector_length(c2)
    
    if l1 > 0 and l2 > 0:
        return dot_product(c1, c2) / (l1 * l2)
    else:
        return False

def get_k_largest(vector: list, k):
    k_largest = []
    while k > 0 and max(vector) != 0:
        largest = max(vector)
        k_largest.append(largest)
        vector[vector.index(largest)] = -999999

        k -= 1
    
    return k_largest


In [11]:
m = len(rating_matrix[0])

M = np.zeros((m, m), dtype=object)      # output is m x m Matrix
for j in range(m):
    for i in range(m):
        if i != j:
            columnI = get_column_vector(rating_matrix, i)
            columnJ = get_column_vector(rating_matrix, j)
            sim = cosine_similarity(columnI, columnJ)
            if (sim):
                M[i][j] = sim
            else:
                M[i][j] = 0

display(M)


array([[0, 0.26238676764419283, 0.5763524126874763, ..., 0,
        0.16265001215808886, 0],
       [0.26238676764419283, 0, 0.23012841872238948, ..., 0, 0,
        0.05363935461248598],
       [0.5763524126874763, 0.23012841872238948, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0.16265001215808886, 0, 0, ..., 0, 0, 0],
       [0, 0.05363935461248598, 0, ..., 0, 0, 0]], dtype=object)

In [12]:
for j in range(m):
    for i in range(m):
        columnI = get_column_vector(M, i)
        k_largest = get_k_largest(columnI, K)
        if M[i][j] not in k_largest:
            M[i][j] = 0

In [13]:
display(M)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

# Prediction

In [14]:
USER = 1066   # User to check
N = 1

## Ratings Prediction

### Get list of rated items

In [15]:
def find_item_neighbors(user, item, rating_matrix, similarity_matrix, K):
    corr_items_similarity = similarity_matrix[item]
    
    user_rated_items = rating_matrix[user]
    rated_item_indexes = []
    
    for item_index in range(len(user_rated_items)):
        
        # User has rated the item
        if user_rated_items[item_index] != 0:
            rated_item_indexes.append(item_index)
    
    filtered_similarity_scores = []
    
    # Get neighbors similarity score
    for item_index in rated_item_indexes:
        if item != item_index:
            filtered_similarity_scores.append(
                (item_index, corr_items_similarity[item_index])
            )
    
    # Sort based on simlarity scores 
    # (tuple is in (item_index, sim_score) format)
    filtered_similarity_scores = sorted(filtered_similarity_scores, 
                                       key=lambda x: x[1], 
                                       reverse=True
                                    )
    
    item_neighbors = []

    if len(filtered_similarity_scores) < K:
        item_neighbors = [sim_item[0] for sim_item in filtered_similarity_scores]
        for i in range(K - len(filtered_similarity_scores)):
            item_neighbors.append(-999)
    
    else:
        # Filter top K similar items
        for i in range(K):
            similar_item = filtered_similarity_scores[i]
            item_neighbors.append(similar_item[0])
        
    return item_neighbors

### Item-KNN

In [16]:
def item_KNN(USER, rating_matrix, user_item_context_encodings, M):
    wanted_user = user_item_context_encodings[0][USER]

    inferred_ratings = []
    user_rated_items = rating_matrix[wanted_user]

    for item in range(len(user_rated_items)):
        rating = user_rated_items[item]
        
        # Item has not been rated
        if rating == 0:
            rating_sum = 0
            neighbor_count = 0
            
            # Find k nearest item neighbors
            nearest_neighbors = find_item_neighbors(
                                    wanted_user,
                                    item,
                                    rating_matrix,
                                    M,
                                    K
                                )
            
            # Nearest neighbors are represented as their indexes
            # in the rating matrix
            for item_neighbor in nearest_neighbors:
                if item_neighbor != -999:
                    neighbor_rating = rating_matrix[wanted_user][item_neighbor]
                    
                    rating_sum += neighbor_rating
                    neighbor_count += 1
                
            inferred_rating = rating_sum / neighbor_count if neighbor_count != 0 else 0
            
            if inferred_rating != 0:
                inferred_ratings.append((item, inferred_rating))
    return inferred_ratings, user_rated_items

In [17]:
def get_mapped_ratings_from_context(inferred_ratings, user_item_context_reverse_encodings, context_T):
    mapped_ratings = []
    for i in inferred_ratings:
        
        # Get actual item encoding (from the cartesian product result)
        item_context = context_T[i[0]]
        
        item = user_item_context_reverse_encodings[1][item_context[0]]
        
        new_entry = [item]
        
        for j in range(1, len(item_context)):
            new_entry.append(item_context[j])
        
        new_entry.append(i[1])
        
        mapped_ratings.append(new_entry)
        
    return mapped_ratings

In [18]:

def get_predictions(mapped_ratings, encoded_df):
    predicted_df = pd.DataFrame(
        mapped_ratings, 
        columns=[
            'Item', 
            *encoded_df.columns[3:], 
            'predicted_rating']
    ).sort_values(by='predicted_rating', ascending=False)
    return predicted_df

## Recommendation Generation

In [19]:
def recommendation_generation(N, predicted_df):
    # Select N items to be recommended
    chosen = []

    for data in predicted_df.iterrows():
        data_tup = tuple(data[1])
        
        if len(chosen) <= N:
            chosen.append(data_tup)

    return chosen

## Rating Prediction Translated Result

In [20]:
def get_mapped_ratings_translated(inferred_ratings, context_T, user_item_context_reverse_encodings):
    mapped_ratings = []
    for i in inferred_ratings:
        
        # Get actual item encoding (from the cartesian product result)
        item_context = context_T[i[0]]
        
        item = user_item_context_reverse_encodings[1][item_context[0]]
        
        new_entry = [item]
        
        # Translate remaining context
        for j in range(1, len(item_context)):
            # 1 --> item
            # >= 2 --> context
            translator = user_item_context_reverse_encodings[j+1]
            new_entry.append(translator[item_context[j]])
        
        # Append the rating
        new_entry.append(i[1]*maximum_rating)
        
        mapped_ratings.append(new_entry)
    
    return mapped_ratings

In [21]:
def get_result(mapped_ratings, encoded_df):   
    res = pd.DataFrame(
        mapped_ratings, 
        columns=[
            'Item', 
            *encoded_df.columns[3:], 
            'predicted_rating']
    ).sort_values(by='predicted_rating', ascending=False)
    return res

## Dataset Merge

In [22]:
witheld = pd.read_csv('witheld_ratings.csv')

In [23]:
display(witheld)

,userid,itemid,rating,Time,Location,Companion
0,1066,tt4411490,1,Weekend,Cinema,Family
1,1066,tt1707386,2,Weekday,Home,Partner
2,1066,tt4411490,1,Weekday,Cinema,Family
3,1066,tt0232500,1,Weekday,Home,Partner
4,1066,tt1707386,1,Weekend,Cinema,Family
...,...,...,...,...,...,...
147,1098,tt0405422,3,Weekend,Cinema,Alone
148,1098,tt0211915,4,Weekend,Home,Family
149,1098,tt0169547,4,Weekend,Cinema,Alone
150,1098,tt0289879,1,Weekday,Home,Alone


In [24]:
def merge_df(res, witheld, USER):
    actual_ratings = []
    for row in res.iterrows():
        row_data = row[1]
        actual = witheld[
            (witheld['userid'] == USER) &\
            (witheld['itemid'] == row_data['Item']) &\
            (witheld['Time'] == row_data['Time']) &\
            (witheld['Location'] == row_data['Location']) &
            (witheld['Companion'] == row_data['Companion'])
        ]['rating']
        
        if(not actual.empty):
            actual_ratings.append(tuple(actual)[0])
            
        else:
            actual_ratings.append(np.nan)   
    

    return actual_ratings  

In [25]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [26]:

sum_rms = 0
actual_users = 0

for USER in user_item_context_encodings[0]:
    N = 10
    inferred_ratings, user_rated_items = item_KNN(USER, rating_matrix, user_item_context_encodings, M)

    mapped_ratings = get_mapped_ratings_from_context(inferred_ratings, user_item_context_reverse_encodings, context_T)
    predicted_df = get_predictions(mapped_ratings, encoded_df)

    chosen = recommendation_generation(N, predicted_df)
    
    mapped_ratings = get_mapped_ratings_translated(inferred_ratings, context_T, user_item_context_reverse_encodings)
    
    res = get_result(mapped_ratings, encoded_df)
    
    actual_ratings = merge_df(res, witheld, USER)
    
    merged_result = res.copy()
    merged_result['actual_rating'] = actual_ratings
    merged_result[(merged_result['actual_rating'] != np.nan)]  
    merged_result.dropna(inplace=True)


    if len(merged_result) != 0:
        actual_users += 1
        rms = mean_squared_error(merged_result['actual_rating'], merged_result['predicted_rating'], squared=False)
        sum_rms += rms


print("actual users: " + str(actual_users))
print(sum_rms / actual_users)
    


actual users: 17
1.2831799545125984
